# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

### Prerequiries

In [ ]:
# !pip install pyquaternion

### Initialization

In [108]:
from bioexplorer import BioExplorer, Protein, AssemblyProtein, Surfactant, Membrane, Virus, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
from pyquaternion import Quaternion as pyQuaterion
from braynsmediamaker import MovieMaker

be = BioExplorer('localhost:5000')
core = be.core_api()
mm = MovieMaker(core)
status = be.reset()

### Resources

In [109]:
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'


glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

### Configuration

In [110]:
# Scene
scene_size = 250.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
# protein_representation = be.REPRESENTATION_CONTOURS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False
nb_defensins_on_virus = 2

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

## Camera and rendering settings

In [111]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

In [167]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 350
params.fog_thickness = 150
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

In [173]:
core.clear_lights()
core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-1,-1,-1],
    intensity=1.0, is_visible=False
)

## Movie

In [145]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 0.0, -1.0],
        'focusDistance': 0.0,
        'origin': [1.6502412168755822, 31.85387269937135, 412.9385526569384],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.456084843117388, -0.027656571035110163, -0.8895064530160348],
        'focusDistance': 0.0,
        'origin': [-6.538792701192037, -65.30405597715314, 144.1078241353965],
        'up': [0.0007301065214662455, 0.9995050281008051, -0.0314510054795716]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 0.0, -1.0],
        'focusDistance': 0.0,
        'origin': [-37.318183489968035, -2.5200681155891576, 274.76964004538524],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.10258399568439694, -0.06406096323380003, -0.9926594163251469],
        'focusDistance': 0.0,
        'origin': [6.6567467250689765, 32.473162591007394, 169.645985506744],
        'up': [-0.13586210720011824, 0.9894745801505731, -0.04981508870804724]
    }
]
mm.build_camera_path(key_frames, 200, 50)

### Cell, Coronavirus and Surfactant-D

In [114]:
def add_virus(
    name, position, orientation=Quaternion(), 
    open_conformation_indices=list(), 
    position_seed=1, orientation_seed=2, 
    morphing_step=0.0):
    
    closed_conformation_indices = list()
    for i in range(nb_protein_s):
        if i not in open_conformation_indices:
            closed_conformation_indices.append(i)

    virus_protein_s = Protein(
        sources=[
            pdb_folder + '6vyb.pdb',  # Open conformation
            pdb_folder + 'sars-cov-2-v1.pdb'  # Closed conformation
        ],
        load_hydrogen=protein_load_hydrogen, occurences=nb_protein_s, 
        assembly_params=[
            11.5,
            position_seed, 0.0,
            orientation_seed, 0.1,
            morphing_step
        ],
        cutoff_angle=0.999,
        orientation=Quaternion(0.0, 1.0, 0.0, 0.0),
        instance_indices=[
            open_conformation_indices, 
            closed_conformation_indices
        ])

    virus_protein_m = Protein(
        sources=[pdb_folder + 'QHD43419a.pdb'], load_hydrogen=protein_load_hydrogen,
        occurences=nb_protein_m, 
        assembly_params=[
            2.5,
            1, 0.1,
            orientation_seed + 2, 0.3,
            morphing_step
        ],
        cutoff_angle=0.999,
        orientation=Quaternion(0.99, 0.0, 0.0, 0.135))

    virus_protein_e = Protein(
        sources=[pdb_folder + 'QHD43418a.pdb'], load_hydrogen=protein_load_hydrogen, 
        occurences=nb_protein_e, 
        assembly_params=[
            3.0,
            3, 0.1,
            orientation_seed + 4, 0.3,
            morphing_step
        ],
        cutoff_angle=0.9999, 
        orientation=Quaternion(0.705, 0.705, -0.04, -0.04))

    virus_membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=10000)

    rna_sequence = None
    clip_planes = list()
    if add_rna:
        clip_planes.append([0, 0, -1, 15])
        import math
        rna_sequence = RNASequence(
            source=rna_folder + 'sars-cov-2.rna',
            assembly_params=[
                11.0, 
                position_seed + 5, 0.0,
                orientation_seed + 6, 0.0,
                morphing_step
            ],
            t_range=Vector2(0, 30.5 * math.pi), 
            shape=be.RNA_SHAPE_TREFOIL_KNOT,
            shape_params=Vector3(1.51, 1.12, 1.93))

    coronavirus = Virus(
        name=name, protein_s=virus_protein_s, 
        protein_e=virus_protein_e,
        protein_m=virus_protein_m, membrane=virus_membrane,
        rna_sequence=rna_sequence,
        assembly_params=[
            45.0, 
            4, 0.05,
            orientation_seed + 7, 0.5,
            morphing_step
        ]
    )

    be.add_virus(
        virus=coronavirus, position=position, orientation=orientation,
        representation=protein_representation, 
        atom_radius_multiplier=protein_radius_multiplier,
        clipping_planes=clip_planes
    )

    if add_glycans:
        # High-mannose
        indices_closed = [61, 122, 234, 603, 709, 717, 801, 1074]
        indices_open = [61, 122, 234, 709, 717, 801, 1074]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=high_mannose_paths, 
            indices=indices_closed, representation=glycan_representation)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=high_mannose_paths,
            indices=indices_open, representation=glycan_representation)

        # Complex
        indices_closed = [17, 74, 149, 165, 282, 331,
                          343, 616, 657, 1098, 1134, 1158, 1173, 1194]
        indices_open = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=complex_paths, 
            indices=indices_closed,
            representation=glycan_representation)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=complex_paths, 
            indices=indices_open, representation=glycan_representation)

        # O-Glycans
        for index in [323, 325]:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_PROTEIN_S_CLOSED, 
                site_indices=[index], representation=glycan_representation)
            be.add_sugars(o_glycan)

        # High-mannose glycans on Protein M
        indices = [5]
        high_mannose_glycans = Sugars(
            orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=name + '_' + be.NAME_PROTEIN_M, source=high_mannose_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(high_mannose_glycans)


        # Complex glycans on Protein E
        indices = [48, 66]
        complex_glycans = Sugars(
            orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
            protein_name=name + '_' + be.NAME_PROTEIN_E, source=complex_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(complex_glycans)
        
    # Stick defensins to virus
    for i in range(nb_defensins_on_virus):
        defensin = AssemblyProtein(
            assembly_name=name, name=name + '_' + be.NAME_DEFENSIN + '_' + str(i),
            source=defensin_path, assembly_params=[0, 0.0, 0, 0.0, 2.0, 0.0], 
            shape=be.ASSEMBLY_SHAPE_SPHERICAL,
            atom_radius_multiplier=protein_radius_multiplier, 
            representation=protein_representation,
            random_seed=100 + i + 1)
        be.add_assembly_protein(defensin)

In [115]:
def add_cell(random_position_seed, random_orientation_seed):
    name='Cell'

    '''ACE2 receptor definition'''
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'],
        occurences=1,
        position=Vector3(0.0, 6.0, 0.0))

    '''Membrane definition'''
    membrane_size = scene_size
    membrane_height = scene_size / 10.0
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=40000)

    '''Cell definition'''
    cell = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=random_position_seed,
        random_position_strength=scene_size / 500.0,
        random_orientation_seed=random_orientation_seed,
        random_orientation_strength=0.5,
        extra_parameters=[membrane_height]
    )

    '''Add cell to scene'''
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=Vector3(0.0, -80.0, 0.0)
    )

    '''Color scheme'''
    be.set_protein_color_scheme(
        name, name + '_' + be.NAME_RECEPTOR, 
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    
    '''Glycans'''
    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name, 
        glycan_type=be.NAME_GLYCAN_COMPLEX,
        protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
        indices=[53, 90, 103, 322, 432, 690])

    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name,
        glycan_type=be.NAME_GLYCAN_HYBRID,
        protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
        indices=[546])

    indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
               [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
    for index in indices:
        o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
        o_glycan = Sugars(
            assembly_name=name, name=o_glycan_name, 
            source=o_glycan_paths[0],
            protein_name=name + '_' + be.NAME_RECEPTOR, 
            representation=protein_representation,
            chain_ids=[2, 4], site_indices=[index[0]], 
            orientation=index[1])
        be.add_sugars(o_glycan)
    return status

In [116]:
def add_surfactant_d(name, position, orientation, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, orientation=orientation, 
        random_seed=random_seed)

In [117]:
def add_glucose(random_seed, nb_glucoses=14400):
    volume_position = Vector3(0.0, scene_size / 2.0 - 80.0, 0.0)
    glucose = Protein(
        sources=[glucose_path], load_non_polymer_chemicals=True, 
        occurences=nb_glucoses)
    volume = Volume(
        name=be.NAME_GLUCOSE, size=scene_size, 
        protein=glucose,
        random_position_seed=random_seed,
        random_position_stength=scene_size / 500.0,
        random_orientation_seed=random_seed,
        random_orientation_stength=0.3
    )
    return be.add_volume(
        volume=volume, 
        representation=be.REPRESENTATION_ATOMS_AND_STICKS,
        atom_radius_multiplier=protein_radius_multiplier,
        position=volume_position)

In [153]:
'''Coronavirus'''
q_c_start = pyQuaterion(0.707, 0.707, 0.0, 0.0)
q_c_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)
q_c_nb_frames = 200

'''Surfactant-D'''
q_s_start = pyQuaterion(0.0, 0.0, 0.707,0.707)
q_s_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)
q_s_nb_frames = 600

In [120]:
add_cell(1,2)
be.set_models_visibility(True)

{'contents': 'OK', 'status': True}

In [162]:
mm.set_current_frame(1)

In [201]:
from IPython.display import clear_output

name='Coronavirus'
output_folder='/home/favreau/Videos/animation/full_scene_binding'
nb_frames = mm.get_nb_frames()

be.set_general_settings(model_visibility_on_creation=False, off_folder='/tmp')
core.set_application_parameters(image_stream_fps=0)

for frame in range(1, nb_frames, 5):
    clear_output()
    print('Frame %i out of %i' % (frame, nb_frames))
    
    '''Camera'''
    if False:
        mm.set_current_frame(frame)
    
    if True:
        '''Cell'''
        add_cell(frame, frame + 1)
    
    if True:
        '''Coronavirus'''
        p = Vector3(-44.25, -1.643, 96.500)
        o = q_c_end
        if frame <= q_c_nb_frames:
            p = Vector3(-44.25, 198.643 - frame, -3.5 + frame / 2.0)
            o = pyQuaterion.slerp(q_c_start, q_c_end, (frame + 1) / q_c_nb_frames)
        
        add_virus(
            name=name, position=p, orientation=Quaternion(o[0],o[1],o[2],o[3]),
            open_conformation_indices=[1], 
            position_seed=frame + 1, orientation_seed=frame + 2
        )            

    if True:
        '''Surfactant-D'''
        o = pyQuaterion.slerp(
            q_s_start, q_s_end, (frame + 1) / q_s_nb_frames)
        add_surfactant_d(
            name='Surfactant-D 1', random_seed=1,
            position=Vector3(
                25.0,
                18.0 + (nb_frames - frame) / 10.0, 
                72.0 + (nb_frames - frame / 2.0) / 10.0),
            orientation = Quaternion(o[0],o[1],o[2],o[3])
        )

    if True:
        '''Glucose'''
        add_glucose(frame)
    
    '''Materials'''
    be.apply_default_color_scheme(
        shading_mode=be.SHADING_MODE_BASIC, user_parameter=1)
    be.set_models_visibility(True)

    if True:
        '''Snapshot'''
        mm.create_snapshot(
            size=[960, 540], samples_per_pixel=32,
            path=output_folder + '/%05d.png' % frame)

core.set_application_parameters(image_stream_fps=20)

Frame 596 out of 601


IntProgress(value=0, max=36)

IntProgress(value=0, description='In progress...')

True